In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
# Imports we'll be using
from tensorflow import keras
from keras.layers import *
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
RunningInCOLAB = 'google.colab' in str(get_ipython())
RunningInCOLAB

True

In [ ]:
path = "./data_collection/training_set_1.csv"
if RunningInCOLAB: path = "/content/drive/MyDrive/24Winter/cs320-w24/cs320-final-project/data_collection/training_set_1.csv"
df_import = pd.read_csv(path)

In [ ]:
df_import.head()

,n,gameOver,head,score,visionUp,visionRight,visionDown,visionLeft,directionChoice
0,0,0,"[8, 9]",0,0.0,0.0,0.0,1.0,3
1,1,0,"[8, 10]",0,0.0,0.0,0.0,1.0,3
2,2,0,"[8, 11]",0,0.0,0.0,0.0,1.0,0
3,3,0,"[7, 11]",0,0.0,0.0,1.0,0.0,2
4,4,0,"[6, 11]",0,0.0,0.0,1.0,0.0,3


In [ ]:
df = df_import[['gameOver','visionUp','visionRight','visionDown','visionLeft','directionChoice']]

In [ ]:
df.head()

,gameOver,visionUp,visionRight,visionDown,visionLeft,directionChoice
0,0,0.0,0.0,0.0,1.0,3
1,0,0.0,0.0,0.0,1.0,3
2,0,0.0,0.0,0.0,1.0,0
3,0,0.0,0.0,1.0,0.0,2
4,0,0.0,0.0,1.0,0.0,3


In [ ]:
dummy_columns = pd.get_dummies(df['directionChoice'], prefix='directionChoice')

In [ ]:
dummy_columns = dummy_columns.replace({True:1, False:0})

In [ ]:
df = pd.concat([df, dummy_columns], axis=1)
df.drop('directionChoice', inplace=True, axis=1)

In [ ]:
# subset for only positive examples
df = df[df['gameOver'] == 0]

In [ ]:
df.head()

,gameOver,visionUp,visionRight,visionDown,visionLeft,directionChoice_0,directionChoice_1,directionChoice_2,directionChoice_3
0,0,0.0,0.0,0.0,1.0,0,0,0,1
1,0,0.0,0.0,0.0,1.0,0,0,0,1
2,0,0.0,0.0,0.0,1.0,1,0,0,0
3,0,0.0,0.0,1.0,0.0,0,0,1,0
4,0,0.0,0.0,1.0,0.0,0,0,0,1


In [ ]:
# X = df[['directionChoice_0','directionChoice_1', 'directionChoice_2', 'directionChoice_3','visionUp', 'visionRight', 'visionDown', 'visionLeft']].to_numpy()
X = df[['visionUp','visionRight','visionDown', 'visionLeft']]
y = df[['directionChoice_0','directionChoice_1','directionChoice_2','directionChoice_3']].to_numpy()

In [ ]:
# X = X[0:30000]
# y = y[0:30000]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# make training data more managable


In [ ]:
# First, we initialize a sequential model
# Create a Sequential model
model = keras.Sequential()

# Add a single Dense layer with 5 input nodes and 1 output node
model.add(Input(shape=(4,)))
model.add(Dense(units=(5), activation='relu'))
model.add(Dense(units=(5), activation='relu'))
model.add(Dense(units=(4), activation='softmax'))

# Compile the model
optimizer = keras.optimizers.Adam(learning_rate=1e-2)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 5)                 25        
                                                                 
 dense_1 (Dense)             (None, 5)                 30        
                                                                 
 dense_2 (Dense)             (None, 4)                 24        
                                                                 
Total params: 79 (316.00 Byte)
Trainable params: 79 (316.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Train model
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

# Eval model
out = model.evaluate(X_train, y_train)
print("Training accuracy:", out)

Epoch 1/10
27536/27536 [==============================] - 100s 4ms/step - loss: 0.5585 - accuracy: 0.2556
Epoch 2/10
27536/27536 [==============================] - 98s 4ms/step - loss: 0.5590 - accuracy: 0.2546
Epoch 3/10
 2829/27536 [==>...........................] - ETA: 1:24 - loss: 0.5603 - accuracy: 0.2550

In [ ]:
model.save("./models/possible_mvp.keras")

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)

63/63 [==============================] - 0s 845us/step - loss: 0.0754 - accuracy: 0.9535


In [ ]:
preds = model.predict(X_test)

63/63 [==============================] - 0s 640us/step


In [ ]:
min(preds)

array([5.9780255e-06], dtype=float32)

In [ ]:
max(preds)

array([0.99758655], dtype=float32)

In [ ]:
preds[y_test == 1]

array([[0.3080828 ],
       [0.5061608 ],
       [0.33071992],
       [0.5061608 ],
       [0.5061608 ],
       [0.34959215],
       [0.9914294 ],
       [0.3080828 ],
       [0.99682575],
       [0.3080828 ],
       [0.5061608 ],
       [0.5397804 ],
       [0.3080828 ],
       [0.33071992],
       [0.5743029 ],
       [0.54725945],
       [0.5061608 ],
       [0.3080828 ],
       [0.5061608 ],
       [0.5397804 ],
       [0.33071992],
       [0.4986248 ],
       [0.99429053],
       [0.5397804 ],
       [0.54725945],
       [0.47120044],
       [0.54725945],
       [0.36240047],
       [0.5061608 ],
       [0.5397804 ],
       [0.5061608 ],
       [0.5397804 ],
       [0.74644107],
       [0.33071992],
       [0.3080828 ],
       [0.5743029 ],
       [0.3080828 ],
       [0.5397804 ],
       [0.5061608 ],
       [0.5061608 ],
       [0.28995204],
       [0.3080828 ],
       [0.3080828 ],
       [0.54725945],
       [0.5397804 ],
       [0.33071992],
       [0.28995204],
       [0.330

In [ ]:
for i in range(0, 4):
    direction = [0, 0, 0, 0]
    direction[i] = 1
    a = np.array(direction + [0,1,1,1])
    print(a)



[1 0 0 0 0 1 1 1]
[0 1 0 0 0 1 1 1]
[0 0 1 0 0 1 1 1]
[0 0 0 1 0 1 1 1]


In [ ]:
input = np.array([1,0,0,0,3])
input = input.reshape((1,5))
model.predict(input)

1/1 [==============================] - 0s 21ms/step


array([[0.00321802]], dtype=float32)

In [ ]:
input

array([[1, 0, 0, 0, 1, 0, 0, 0]])

In [ ]:
len(df[(df['visionUp'] == 1) & (df['gameOver'] == 1)])

22283